### A quick note on Python <> Julia integration
The below script relies on Matplotlib and PyTorch being available in the Conda environment attached to Julia. If you don't tend to use Julia very much, you may want to attach one of your favourite Python environments to Julia to avoid reinstalling a bunch of stuff. To do this, run the following commands with the appropriate Python executable referenced. (If PyCall is not yet installed, you could use `pkg"add PyCall"` instead of `build`.

```julia
ENV["PYTHON"]=expanduser("~/anaconda3/envs/mkl/bin/python")  # change path here
using Pkg
pkg"build PyCall"
```
Thanks to [Przemyslaw Szufel](https://stackoverflow.com/a/63182917) for this notebook friendly route.

In [2]:
using LinearAlgebra, Statistics, StatsBase, Random
using PyPlot
using BSON, NPZ, CSV, Tables

using MeshCat
using Quaternions, GeometryBasics, CoordinateTransformations

In [5]:
# Project-specific Julia code
include("viz/mocap_viz.jl")
include("viz/expmtdata.jl")
include("viz/geom3d.jl")
include("viz/prettytbl.jl")
include("viz/util.jl")

Main.mocaputil

## Load in data

Recall that the 30fps processed data can be downloaded from [https://bit.ly/38x2sra](https://bit.ly/38x2sra). Once downloaded, please store the `.npz` files in the `data/` directory in the base project (parent) folder.

In [6]:
# database = "../../mocap-mtds-macbook/data/edin-style-transfer/"
# files_edin = [joinpath(database, f) for f in readdir(database)];
# style_name_edin = [x[1] for x in match.(r"[a-z\-]+/[a-z\-]+/([a-z]+)_.*", files_edin)];
# styles = unique(style_name_edin)
# styles_lkp = [findall(s .== style_name_edin) for s in styles]
styles_lkp = [
    [1, 2, 3, 4, 5, 6],
    [7, 8, 9, 10],
    [11, 12, 13],
    [14, 15, 16, 17],
    [18, 19, 20, 21],
    [22, 23, 24, 25],
    [26, 27, 28],
    [29, 30, 31]
]

8-element Vector{Vector{Int64}}:
 [1, 2, 3, 4, 5, 6]
 [7, 8, 9, 10]
 [11, 12, 13]
 [14, 15, 16, 17]
 [18, 19, 20, 21]
 [22, 23, 24, 25]
 [26, 27, 28]
 [29, 30, 31]

**Do not perform the following cell.**

We need to invert the standardization performed to create the data files used to train/test the model. This
has been saved in BSON format, but the original code to do this is below.

In [7]:
# # Load in data
# data_path2 = "../../mocap-mtds-macbook/"
# Usraw1 = BSON.load(joinpath(data_path2, "edin_Xs_30fps_final.bson"))[:Xs];
# Ysraw1 = BSON.load(joinpath(data_path2, "edin_Ys_30fps_final.bson"))[:Ys];

# standardize_Y1 = fit(mocaputil.MyStandardScaler, reduce(vcat, Ysraw1),  1)
# standardize_U1 = fit(mocaputil.MyStandardScaler, reduce(vcat, Usraw1),  1)

# BSON.bson("../data/standardization_Y.bson", Dict(
#     "μ"=>standardize_Y1.μ, 
#     "σ"=>standardize_Y1.σ, 
#     "operate_on"=>standardize_Y1.operate_on,
#     "dims"=>standardize_Y1.dims
# ))
# BSON.bson("../data/standardization_U.bson", Dict(
#     "μ"=>standardize_U1.μ, 
#     "σ"=>standardize_U1.σ,
#     "operate_on"=>standardize_U1.operate_on,
#     "dims"=>standardize_U1.dims
# ))

In [8]:
# # Load in data
data_path = "../data/"

#= Load from Numpy Compressed files dumped from python pre-processing =#
Ys = npzread(joinpath(data_path, "edin_Ys_30fps_final.npz"))
Us = npzread(joinpath(data_path, "edin_Us_30fps_final.npz"))

# string-based dict to standard (int-based) array
Ys = [Ys[string(i)] for i in 1:31]
Us = [Us[string(i)] for i in 1:31];

In [9]:
standardize_Y = mocaputil.MyStandardScaler(
    let d=BSON.load("../data/standardization_Y.bson"); d["μ"], d["σ"], d["operate_on"], d["dims"]; end...
)
standardize_U = mocaputil.MyStandardScaler(
    let d=BSON.load("../data/standardization_U.bson"); d["μ"], d["σ"], d["operate_on"], d["dims"]; end...
);

In [10]:
this_expmtdata = expmtdata.ExperimentData(
    [zeros(Float32, size(y,1)+1, size(y,2)) for y in Ys],    # Ysraw -- legacy, not needed
    [Matrix(y') for y in Ys], 
    [Matrix(u') for u in Us], 
    styles_lkp
);

----------------------
## Training / test data for models (see note below)
**This section is not specifially useful here**; but this is for demonstration purposes. In my experimental work I saved
these batched training/test sets as `.npz` files and used them to feed to the PyTorch model in a training shell script.

In [69]:
?expmtdata.get_data

```
get_data(s::ExperimentData, ix, splittype, tasktype)
```

Convenience utility for accessing data stored in an ExperimentData struct. Specify the index of the target task, and then select from:

splittype:

  * **:all**        - return the concatentation of all training/validation/test data.
  * **:trainvalid** - return the concatentation of all training/validation data.
  * **:split**      - return individual (3x) outputs for training/validation/test data.
  * **:test**       - return only the test data
  * **:train**      - return only the train data
  * **:valid**      - return only the validation data.

tasktype:

  * **:stl**   - single task model. Return train/validation/test data from this task's data.
  * **:pool**  - pooled/cohort model. Here, training and validation data are from the        complement of the selected index, returned in individual wrappers.

Note that in all cases, the output will be (a) Dict(s) containing the following fields:

  * **:Y**    - the observation matrix (each column is an observation).
  * **:U**    - the input matrix (each column is a datapoint).
  * **:Yraw** - the raw data before standardisation and another manipulation. (Possibly legacy?)

Othe kwargs:

  * `concat`  - By default, each boundary encountered between files will result in

a separate Dict, so the return values will be a vector of Dicts. However, for more basic models (such as linear regression) with no assumption of temporal continuity, it may be simpler to operate on a standard input and output data matrix. Setting `concat = true` will return just a single Dict in an array with all the data. Choosing `simplify=true` will further remove the array, returning only Dicts.

  * `stratified`  - (STL only) stratify the validation/test sets across files in

each style. By default, the test set will come at the end of the concatenation of all files. Stratifying will mean there are L test sets from each of L files. For the pooled dataset, the test set is partially stratified, that is, it is stratified over the *types* (i.e. a % of each style), but not over the *files* within the types. Given that our goal is MTL, this seems appropriate.

  * `split`  - The train/validation/test split as a simplicial 3-dim vector.
  * `simplify` - See `concat`. Used without `concat` this option does nothing.


In [11]:
# Get training set for STL and pooled models.
style_ix = 1                         # which style is to be held out for test data
train_ixs = setdiff(1:8, style_ix) 
min_size = 63;
batch_size = 64;

trainPool, validPool, testPool = expmtdata.get_data(this_expmtdata, style_ix, :split, :pooled)
trainIter = mocaputil.DataIterator(trainPool, 64, min_size=min_size);
trainIters = collect(trainIter);

In [12]:
testIter = mocaputil.DataIterator(testPool, 64, min_size=min_size);
testIters = collect(testIter);

#### What does this training data look like?
Each batch is defined here to have length 64, and hence:

In [13]:
println("Each batch b typically has Y_b of size (n_y, T): $(size(trainIters[1][1]))")
println("                           U_b of size (n_u, T): $(size(trainIters[1][2]))")
println("  and an indicator of whether we have transitioned to a new sequence since the last batch (bool)")

Each batch b typically has Y_b of size (n_y, T): (67, 64)
                           U_b of size (n_u, T): (35, 64)
  and an indicator of whether we have transitioned to a new sequence since the last batch (bool)


#### How many batches do each style have?
And when are their transitions throughout the batched data?

In [14]:
segment_lens = [length(mocaputil.DataIterator(expmtdata.get_data(this_expmtdata, i, :train, :stl, split=[0.875,0.125]),
            64, min_size=63)) for i in train_ixs];
segment_lkp = [collect(i+1:j) for (i,j) in zip(vcat(0, cumsum(segment_lens[1:end-1])), cumsum(segment_lens))];
segment_names = ["angry", "childlike", "depressed", "neutral", "old", "proud", "sexy", "strutting"][train_ixs];
prettytbl.table(reshape(cumsum(vcat(1, segment_lens)[1:end-1]), :, 1), header_col=segment_names, dp=0)

childlike1
depressed117
neutral215
old327
proud426
sexy505
strutting642


------------------
### Now back to the task at hand

## Generating animations from this data

The `file_offsets` describe the difference in position and rotation between the original and smoothed **path** trajectories.

(To emphasise - only the path has had smoothing applied to it.) 

In order to make sure the raw data and the modelled data do not get out of sync (where the latter is the result of inputting the smoothed trajectory) we need to make sure the reconstruction of the modelled data takes this difference into account. Hence there's a bookkeeping headache of keeping such `file_offsets` around for each sequence.

The following code ensures that the Forward Kinematics applied to the smoothed path is in sync with the reconstruction.

In [15]:
file_offsets = CSV.File("../data/file_offsets_pathsmooth.csv") |> Tables.matrix;

**Aside**: we don't get data from the training/test split since these are batched into length-64 sequences, which are only ~ 2 seconds long. In order to get a longer animation we take from the `expmtdata` directly. Alternatively one could do something like the following:

```julia
# Recall 
# * :all - return the concatentation of all training/validation/test data.
# * :stl - return all the data for the chosen style ix (in this case 7)
alldata7 = mocapio.get_data(expmtdata, 7, :all, :stl)
c_U = alldata7[3][:U][:,1:500]       # 3rd file, 1:500 seq ixs
c_Y_tf = alldata7[3][:Y][:,1:500]
```

In [17]:
file_ix = 3  # file_ix ∈ [1,..,31]
c_U = this_expmtdata.Us[file_ix]
c_Y_tf = this_expmtdata.Ys[file_ix]
c_Urecon = mocaputil.invert(standardize_U, Matrix(c_U'));
c_Yrecon = mocaputil.invert(standardize_Y, Matrix(c_Y_tf'));

seq_ixs = 10:600  # which elements of the sequence to visualize
c_path_fk = geom.fk_path(c_Yrecon, seq_ixs, file_offsets[file_ix,:]);

## three.js visualization


Now to actually perform the visualization. This relies on `three.js` and `MeshCat.jl`. The library code included above is my attempt to map a collection of circles and cylinders to construct a skeletal visualization within this environment. This is both not difficult and not trivial. While the original code works ok modulo 2-3 tweaks due to changes in MeshCat's API, unfortunately the angle of the cylinders (i.e. bones) is slightly off in each frame. I think this is just a minor bug in `mocapviz`, but tracking it down is a bit of a pain, so in the interests of time, the visualization has this known defect.

In [19]:
# Open three.js visualization in browser if doesn't exist already
!(@isdefined vis) && begin; vis = Visualizer(); open(vis); end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/alexbird/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


Process(`xdg-open http://127.0.0.1:8700`, ProcessExited(0))

Opening in existing browser session.


In [20]:
vis = mocapviz.create_animation([geom.reconstruct_modelled(c_Yrecon[seq_ixs,:])], 
    "test"; vis=vis, linemesh=[mocapviz.yellowmesh], camera=:back, path = c_path_fk)

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

---------------

## Creating animations comparing model with ground truth

In [5]:
using PyCall

pysys = pyimport("sys")
pytorch = pyimport("torch")
pushfirst!(PyVector(pysys."path"), normpath(pwd(), "../src"));
pymt = pyimport("forjulia") 

PyObject <module 'forjulia' from '/home/alexbird/Documents/phd-work/pytorch-mtds-mocap/src/forjulia.py'>